In [1]:
import requests
import json

## Getting the results through a query

In [99]:
def send_query( executer_url , user, query):
    data = dict(query=query )
    url = '%s/query' % executer_url 
    response=requests.post(url, json=data, auth=(user, 'pass'))
    if response.status_code != 200:
        raise Exception (response)
    else:
        r = json.loads(response.text)
        return r

In [283]:
executer='http://diascld10.epfl.ch:54321'
user = 'federation'
query = """
select distinct patient_id,( select variable_name as name, value from partition) as vars
    from exam_value
    where 
         variable_name = "DX_bl" or
         variable_name = "CDRSB_bl" or
         variable_name = "MMSE_bl" or
         variable_name = "AGE" or
         variable_name = "PTGENDER" or
         variable_name = "PTEDUCAT" 
    group by patient_id
"""
result = send_query(executer, user,query )

In [284]:

empty=set()
for p in result['output']:
    for v in p['vars']:
        if v['value'] == '':
            empty.add( v['name'])
            
for name in empty:
    print name

## Getting the list of different types of diagnosis (DX_bl)

In [285]:
user = 'federation'
query = """
select distinct value
    from exam_value
    where  variable_name = "DX_bl"
"""

result2 = send_query(executer, user,query )
result2

{u'compilationTime': 278,
 u'executionTime': 188,
 u'output': [u'CN', u'LMCI', u'AD', u'EMCI', u'SMC']}

In [286]:
#creating helper dictionary to map diagnosis to a float value
dx= dict()
for n,d in enumerate(result2['output']):
    dx[d]=float(n)
    print d ,"->" ,dx[d]

CN -> 0.0
LMCI -> 1.0
AD -> 2.0
EMCI -> 3.0
SMC -> 4.0


## we will try the random forest classifier
this is as see in example 
http://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances.html

In [287]:
import numpy as np
import sklearn as sk
import sklearn.datasets as skd
import sklearn.ensemble as ske
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

### We need to transform all values to float for the algorithm

In [321]:
import random

def get_dx_value(value):
    #return dx[value]
    if value == 'AD':
        return 1.0
    else:
        return 0.0

random.seed()
data = result['output']
trainX = []
trainY = []
for p in data:
    values = dict()
    y = None
    # transforms vars to a dict and converts the values to float
    for v in p['vars']:
        name =  v['name']
        value = v['value']
        if name == 'DX_bl':
            #y value for training 
            y = dx[value]
        elif  name != 'PTGENDER':
            try:
                values[name] = float(value)
            except:
                print "could not transform %s->'%s'" % (name, value)
                raise
        elif value == 'Male':
            values[name] = 1.0
        else:
            values[name] = 0.0
    
    values['Random'] = 100*random.random()
    # sorts the keys so that all values always in the same order
    keys = values.keys()
    keys.sort()
    trainX.append([values[k] for k in keys])
    trainY.append(y)

In [322]:
keys

[u'AGE', u'CDRSB_bl', u'MMSE_bl', u'PTEDUCAT', u'PTGENDER', 'Random']

In [323]:
forest = ske.ExtraTreesClassifier()
forest.fit(trainX, trainY)

fet_ind = np.argsort(forest.feature_importances_)[::-1]
fet_imp = forest.feature_importances_[fet_ind]

for n, idx in enumerate(fet_ind):
    print "%d,%s,%f " % (n,keys[idx], fet_imp[n])

0,CDRSB_bl,0.375924 
1,MMSE_bl,0.179195 
2,AGE,0.168738 
3,Random,0.150175 
4,PTEDUCAT,0.105422 
5,PTGENDER,0.020546 


In [324]:
forest.feature_importances_

array([ 0.16873786,  0.37592392,  0.17919516,  0.10542175,  0.02054645,
        0.15017487])

### This is using a random forest classifier

In [330]:
reg = ske.RandomForestClassifier()

reg.fit(trainX, trainY);

fet_ind = np.argsort(reg.feature_importances_)[::-1]
fet_imp = reg.feature_importances_[fet_ind]

for n, idx in enumerate(fet_ind):
    print "%d,%s,%f " % (n,keys[idx], fet_imp[n]) 

0,CDRSB_bl,0.364200 
1,MMSE_bl,0.189328 
2,AGE,0.182802 
3,Random,0.157334 
4,PTEDUCAT,0.080726 
5,PTGENDER,0.025610 


### This is using a random forest regressor instead of a ExtraTreesClassifier

In [331]:
reg = ske.RandomForestRegressor()

reg.fit(trainX, trainY);

fet_ind = np.argsort(reg.feature_importances_)[::-1]
fet_imp = reg.feature_importances_[fet_ind]

for n, idx in enumerate(fet_ind):
    print "%d,%s,%f " % (n,keys[idx], fet_imp[n]) 

0,AGE,0.325238 
1,Random,0.239127 
2,CDRSB_bl,0.215509 
3,PTEDUCAT,0.096133 
4,MMSE_bl,0.086882 
5,PTGENDER,0.037111 


In [328]:
l = [a[4] for a in trainX]

print reduce(lambda x, y: x + y, l) / len(l)

0.568221070812
